In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from stitchnet import get_cache_folder, generate

In [3]:
from stitchnet import load_hf_dataset
# load the beans dataset from huggingface
dataset_train, dataset_val = load_hf_dataset('beans', train_split='validation', val_split='test', label_column='labels', seed=47)

Found cached dataset beans (/home/jupyter-steerapi/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791)


  0%|          | 0/3 [00:00<?, ?it/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Loading cached shuffled indices for dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791/cache-cac060a5360da545.arrow
Loading cached processed dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791/cache-5f0b2533f6c38720.arrow
Loading cached processed dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791/cache-940888201d588dc5.arrow
Loading cached shuffled indices for dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791/cache-fb581327019d6d2b.arrow
Lo

In [4]:
import numpy as np
from tqdm import tqdm
stitching_dataset = np.vstack([x['pixel_values'] for x in tqdm(dataset_train.select(range(32)))])
score,net = generate(stitching_dataset, threshold=0, totalThreshold=0, maxDepth=10, K=2, sample=True)
print(score, net)

100%|██████████| 5/5 [00:00<00:00, 6230.40it/s]


['/home/jupyter-steerapi/.cache/stitchnet/_models/alexnet.onnx', '/home/jupyter-steerapi/.cache/stitchnet/_models/densenet121.onnx', '/home/jupyter-steerapi/.cache/stitchnet/_models/mobilenet_v3_small.onnx', '/home/jupyter-steerapi/.cache/stitchnet/_models/resnet50.onnx', '/home/jupyter-steerapi/.cache/stitchnet/_models/vgg16.onnx']


5it [00:00, 4738.26it/s]
5it [00:01,  4.44it/s]
/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:65: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


potential next fragments before thresholding of 0: 2 ['1.0', '0.75']
potential next fragments after thresholding of 0: 1 ['0.75']
totalscore before thresholding of 0: 0.7485116720199585
potential next fragments before thresholding of 0: 2 ['0.93', '0.7']
potential next fragments after thresholding of 0: 1 ['0.93']
totalscore before thresholding of 0: 0.6936578094513095
potential next fragments before thresholding of 0: 2 ['0.96', '0.76']
potential next fragments after thresholding of 0: 1 ['0.96']
totalscore before thresholding of 0: 0.6687199394456858
potential next fragments before thresholding of 0: 2 ['0.99', '0.77']
potential next fragments after thresholding of 0: 1 ['0.77']
totalscore before thresholding of 0: 0.5129298499647756


In [11]:
net.get_macs_params()

{'macs': 4488343528.0, 'params': 25653096}

In [12]:
net.save_onnx('./_results/net')

saving to ./_results/net.onnx


In [13]:
net.draw_svg('./_results/net')

saving to ./_results/net.svg


In [10]:
net.fit(dataset_train, label_column="labels")

/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:65: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(
100%|██████████| 5/5 [00:01<00:00,  4.05it/s]


In [11]:
for item in dataset_val:
    print(item['pixel_values'].shape)
    break

torch.Size([1, 3, 224, 224])


In [12]:
net.evaluate_dataset(dataset_val, label_column='labels')

100%|██████████| 4/4 [00:01<00:00,  2.04it/s]


{'accuracy': 0.7421875}

In [13]:
net.predict_files(['./_data/test.jpg'])

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


[{'score': [0.2, 0.6, 0.2], 'label': 1}]